# Quick And Dirty Sim Results Extraction

Need to loop through all of the dynamics sequences and extract pickled dictionaries just like the SMPL pipeline would return.

In [1]:
import os
import sys
from pathlib import Path
import pickle

import bpy
import numpy as np

sys.path.append("..")
sys.path.append("../../")
from blender_util.mesh import mesh_to_numpy
from blender_util_dylan.gripper import GripperData

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

In [2]:
FILE_ROOT = Path(os.getcwd())
print("File root:", FILE_ROOT)
GARMENTNETS_ROOT = (FILE_ROOT / ".." / "..").absolute().resolve()
print("GarmentNets root:", GARMENTNETS_ROOT)
SIM_DATASET_DIR = (FILE_ROOT / ".." / "script_output" / "full_dataset_attempt_2").absolute()
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))

ZARR_FILEPATH = GARMENTNETS_ROOT / "data" / "garmentnets_tracking_simulation_dataset.zarr"

DYNAMICS_SEQUENCE_FILENAME_TEMPLATE = "dynamics_seq_{idx}"
CAMERA_IDX_USED_FOR_ANIMATION = 0
NUM_ACTION_SEQUENCES_PER_SAMPLE = 5

File root: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/data_packager
GarmentNets root: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking


In [8]:
frame_start = 0
pickle_name = "dynamics_sim_results.pkl"
for sample_dir in SIM_DATASET_DIR.iterdir():
    print(sample_dir.name)
    for i in range(NUM_ACTION_SEQUENCES_PER_SAMPLE):
        print(f"\tAction seq #{i}")
        seq_dir = sample_dir / f"dynamics_seq_{i}"
        blend_file = seq_dir / "dynamics_animation.blend"
        control_seq_file = seq_dir / "control_sequence.pkl"

        # Read the control sequence file
        control_seq = pickle.load(control_seq_file.open('rb'))
        frame_end = frame_start
        for s in control_seq:
            frame_end += s["frame_duration"]
        # print(frame_end)
        
        bpy.ops.wm.open_mainfile(filepath=blend_file.as_posix())

        # Use same method as SMPL to convert mesh to numpy for all time steps.
        cloth_obj = bpy.data.objects["cloth"]
        cloth_state_keys = ['verts', 'edges', 'faces', 'uv_verts', 'uv_faces']
        cloth_states_list = []
        for j in range(frame_start, frame_end + 1):
            bpy.context.scene.frame_set(j)
            cloth_states_list.append(mesh_to_numpy(cloth_obj, global_coordinate=True))
        
        cloth_states = dict()
        for k in cloth_state_keys:
            cloth_states[k] = np.stack([c[k] for c in cloth_states_list], axis=0)
        
        # Get information on the grippers.
        gripper_data = GripperData()


        # Dump information into the results dictionary.
        results_dict = {
            "gripper_data": {
                "locations": gripper_data.gripper_locations,
                "velocities_meters_per_second": gripper_data.velocity_meters_per_second
            },
            "simulation_info": {
                "frame_start": frame_start,
                "frame_end": frame_end
            },
            "cloth_state": cloth_states
        }
        pickle_file_path = seq_dir / pickle_name
        pickle.dump(results_dict, pickle_file_path.open('wb'))
        # print(gripper_data.gripper_locations)
        # print(gripper_data.velocity_meters_per_second)
        # print(results_dict)




00471_Tshirt_153
	Action seq #0
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/data_packager/../script_output/full_dataset_attempt_2/00471_Tshirt_153/dynamics_seq_0/dynamics_animation.blend
WARNING! Hardcoded number of images rendered, effectively capping at 200 frames.
	Action seq #1
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/data_packager/../script_output/full_dataset_attempt_2/00471_Tshirt_153/dynamics_seq_1/dynamics_animation.blend
WARNING! Hardcoded number of images rendered, effectively capping at 200 frames.
	Action seq #2
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/data_packager/../script_output/full_dataset_attempt_2/00471_Tshirt_153/dynamics_seq_2/dynamics_animation.blend
WARNING! Hardcoded number of images rendered, effectively capping at 200 frames.
	Action seq #3
Read blend: /home/dcolli23/code/school/rob59